**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [2]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization

Using TensorFlow backend.


In [3]:
import skvideo
skvideo.setFFmpegPath("/Users/Antonin/Desktop/ffmepg/")
import skvideo.io

# Check that FFmpeg has been found
print(skvideo.getFFmpegPath())
print(skvideo.getFFmpegVersion())

/Users/Antonin/Desktop/ffmepg/
b'4'.b'1-tessus'.0


# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [9]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [10]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

**If the agent sticks to a suboptimal policy, he might not explore other paths that would allow him to improve its policy. Taking random actions with a non-zero probability ensures that the agents continues to explore the environment, which will allow it to find an optimal policy.**

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [11]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [5]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=100 # set small when debugging
epochs_test=11 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

**The array** ```position``` **refers to the position of the agent in the grid, to make sure that it can not go out of the frame.**

**The array** ```board``` **encompasses the positions of the bonus and malus of the environment.**

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [267]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        return np.random.randint(0, self.n_action)

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [255]:
def test(agent,env,epochs,prefix='', prt=True):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)


        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose
        if prt:
            print("Win/lose count {}/{}. Average score ({})"
                  .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [18]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='videos/random')
HTML(display_videos('videos/random0.mp4'))

Win/lose count 7.0/16.0. Average score (-9.0)
Win/lose count 8.0/18.0. Average score (-9.5)
Final score: -9.5


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




**Answer**
We have,
$$
\begin{align*}
Q^\pi(s,a) &= \mathbb{E}_\pi \left[\sum_{k=0}^{\infty} \gamma^k r(s_k,a_k) | s_0 = s, a_0 = a \right] \\
&= r(s,a) + \mathbb{E}_{\pi} \left[ \sum_{k=1}^{\infty} \gamma^k r(s_k, a_k) | s_0 = s, a_0 = a \right] \\ 
&= r(s,a) + \gamma \sum_{(s',a') \in \mathcal{S} \times \mathcal{A}} p_\pi(s_1 = s', a_1 = a' | s_0 = s, a_0 = a) 
\mathbb{E}_\pi\left[ \sum_{k=0}^{\infty} \gamma^k r(s_{k+1}, a_{k+1}) | s_1 = s', a_1 = a' \right] \\
&= r(s, a) + \gamma \mathbb{E}_{(s',a')\sim \pi(.|s,a)} \left[ Q^\pi (s',a') \right]
\end{align*}$$

Then,
$$
\begin{align*}
Q^*(s,a) &= \max_{\pi} Q^\pi (s,a) \\
&= \max_{\pi} \mathbb{E}_{(s',a')\sim \pi(.|s,a)} \left[ r(s,a) + \gamma Q^\pi (s', a') \right] \\
&= r(s,a) + \gamma \max_\pi \mathbb{E}_\pi [Q^\pi(s',a')] \\
&= r(s,a) + \gamma \max_{\pi,a'} \sum_{s'\in \mathcal{S}} p_\pi(s',a' | s,a) Q^\pi(s',a') \\
&= r(s,a) + \gamma  \sum_{s'\in \mathcal{S}} \pi^*(s'|s,a) \max_{a'} Q^*(s',a') \\
&= r(s,a) + \gamma \mathbb{E}_{s'\sim \pi^*(.|s,a)} \left[ \max_{a'} Q^*(s',a') \right] \\
&= \mathbb{E}_{s'\sim \pi^*(.|s,a)} \left[ r(s,a) + \gamma  \max_{a'} Q^*(s',a') \right]
\end{align*}
$$

Finally, since the function 
$$\theta \mapsto E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}$$
is positive and equal to zero if and only if $Q(s,a,\theta) = Q^*(s,a)$, the loss 
$$\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}$$ is a plausible objective function.

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [268]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        if len(self.memory) < self.max_memory:
            # While the memory is not full, we simply add the move
            self.memory.append(m)
        else:
            # If memory full, we replace an old memory at random
            i = np.random.randint(0, len(self.memory))
            self.memory[i] = m
            
    def random_access(self):
        i = np.random.randint(0, len(self.memory))
        return self.memory[i]


***
The pipeline we will use for training is given below:

In [166]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [227]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        return np.argmax(self.model.predict(s.reshape(1, 5, 5, self.n_state)))
    
    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            #memory = self.memory.random_access()
            s, n_s, a, r, game_over = self.memory.random_access()
            if game_over_:
                target_q[i, a] = r
            else:
                target_q[i, a] = r + self.discount*np.max(self.model.predict(n_s.reshape(1, 5, 5, self.n_state)).reshape(-1))
            
            input_states[i] = s
        
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)

        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        # We use a simple neural network with one fully connected layer
        model = keras.Sequential()
        model.add(Reshape((-1,), input_shape=(5, 5, self.n_state)))
        model.add(Dense(4))
        
        # Optimize with standard SGD with a decay to lower the learning rate during training
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        

In [240]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.05, epsilon = 0.1, memory_size=3000, batch_size = 16)
train(agent, env, epochs_train, prefix='videos/fc_train')
HTML(display_videos('videos/fc_train10.mp4'))

Epoch 000/100 | Loss 0.1174 | Win/lose count 13.5/14.0 (-0.5)
Epoch 001/100 | Loss 0.0269 | Win/lose count 2.5/5.0 (-2.5)
Epoch 002/100 | Loss 0.0814 | Win/lose count 7.0/8.0 (-1.0)
Epoch 003/100 | Loss 0.0097 | Win/lose count 4.5/4.0 (0.5)
Epoch 004/100 | Loss 0.0203 | Win/lose count 3.5/6.0 (-2.5)
Epoch 005/100 | Loss 0.0082 | Win/lose count 10.5/6.0 (4.5)
Epoch 006/100 | Loss 0.0075 | Win/lose count 7.5/6.0 (1.5)
Epoch 007/100 | Loss 0.0317 | Win/lose count 10.5/5.0 (5.5)
Epoch 008/100 | Loss 0.0236 | Win/lose count 9.5/8.0 (1.5)
Epoch 009/100 | Loss 0.0392 | Win/lose count 5.5/7.0 (-1.5)
Epoch 010/100 | Loss 0.0258 | Win/lose count 8.0/1.0 (7.0)
Epoch 011/100 | Loss 0.0230 | Win/lose count 7.5/0 (7.5)
Epoch 012/100 | Loss 0.0338 | Win/lose count 2.5/3.0 (-0.5)
Epoch 013/100 | Loss 0.0203 | Win/lose count 13.5/4.0 (9.5)
Epoch 014/100 | Loss 0.0073 | Win/lose count 4.5/0 (4.5)
Epoch 015/100 | Loss 0.0047 | Win/lose count 14.5/10.0 (4.5)
Epoch 016/100 | Loss 0.0066 | Win/lose count 8.

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [243]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        # We use a CNN with two convlutional layers and one fully connected layer
        model = keras.Sequential()
        model.add(Conv2D(32, (3, 3), input_shape=(5, 5, self.n_state)))
        model.add(Conv2D(32, (2, 2)))
        model.add(Reshape((-1, )))
        model.add(Dense(4))
        
        # Optimize with standard SGD with a decay to lower the learning rate during training
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")

        self.model = model

In [248]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.05, epsilon = 0.1, memory_size=3000, batch_size = 16)
train(agent,env,epochs_train,prefix='videos/cnn_train')
HTML(display_videos('videos/cnn_train10.mp4'))

Epoch 000/100 | Loss 0.0059 | Win/lose count 2.5/6.0 (-3.5)
Epoch 001/100 | Loss 0.0040 | Win/lose count 5.0/3.0 (2.0)
Epoch 002/100 | Loss 0.0199 | Win/lose count 6.5/5.0 (1.5)
Epoch 003/100 | Loss 0.0183 | Win/lose count 5.0/10.0 (-5.0)
Epoch 004/100 | Loss 0.0070 | Win/lose count 4.5/5.0 (-0.5)
Epoch 005/100 | Loss 0.0057 | Win/lose count 8.5/7.0 (1.5)
Epoch 006/100 | Loss 0.0058 | Win/lose count 3.5/0 (3.5)
Epoch 007/100 | Loss 0.0040 | Win/lose count 4.0/7.0 (-3.0)
Epoch 008/100 | Loss 0.0016 | Win/lose count 6.0/4.0 (2.0)
Epoch 009/100 | Loss 0.0019 | Win/lose count 11.5/5.0 (6.5)
Epoch 010/100 | Loss 0.0178 | Win/lose count 12.0/2.0 (10.0)
Epoch 011/100 | Loss 0.0030 | Win/lose count 8.0/3.0 (5.0)
Epoch 012/100 | Loss 0.0039 | Win/lose count 1.0/3.0 (-2.0)
Epoch 013/100 | Loss 0.0042 | Win/lose count 9.5/2.0 (7.5)
Epoch 014/100 | Loss 0.0020 | Win/lose count 6.0/7.0 (-1.0)
Epoch 015/100 | Loss 0.0161 | Win/lose count 11.5/2.0 (9.5)
Epoch 016/100 | Loss 0.0050 | Win/lose count 2.

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [249]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='models/cnn_trainmodel.h5',name_model='models/cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='models/fc_trainmodel.h5',name_model='models/fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='models/cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='models/fc_test')

Test of the CNN
Win/lose count 6.0/4.0. Average score (2.0)
Win/lose count 2.0/1.0. Average score (1.5)
Win/lose count 3.5/2.0. Average score (1.5)
Win/lose count 1.5/1.0. Average score (1.25)
Win/lose count 9.5/1.0. Average score (2.7)
Win/lose count 6.0/2.0. Average score (2.9166666666666665)
Win/lose count 9.5/3.0. Average score (3.4285714285714284)
Win/lose count 5.5/0. Average score (3.6875)
Win/lose count 2.5/2.0. Average score (3.3333333333333335)
Win/lose count 5.5/3.0. Average score (3.25)
Win/lose count 9.5/5.0. Average score (3.3636363636363638)
Final score: 3.3636363636363638
Test of the FC
Win/lose count 11.0/15.0. Average score (-4.0)
Win/lose count 2.5/4.0. Average score (-2.75)
Win/lose count 2.5/0. Average score (-1.0)
Win/lose count 4.5/2.0. Average score (-0.125)
Win/lose count 3.0/1.0. Average score (0.3)
Win/lose count 4.5/4.0. Average score (0.3333333333333333)
Win/lose count 2.5/0. Average score (0.6428571428571429)
Win/lose count 7.0/6.0. Average score (0.6875)


**As we could expect, the DQN with a CNN performs better than the fully connected DQN**

In [250]:
HTML(display_videos('videos/cnn_test10.mp4'))


In [251]:
HTML(display_videos('videos/fc_test10.mp4'))


**An issue that we can observe is that the agent often gets stuck in a loop, visiting the same empty cells over and over.**

### Test for different temperatures

In [257]:
temperatures = [0.3, 0.6, 0.9]
for t in temperatures:
    env = Environment(grid_size=size, max_time=T,temperature=t)
    agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
    agent_cnn.load(name_weights='models/cnn_trainmodel.h5',name_model='models/cnn_trainmodel.json')

    agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
    agent_cnn.load(name_weights='models/fc_trainmodel.h5',name_model='models/fc_trainmodel.json')
    print('Test of the CNN - Temperature : {}'.format(t))
    test(agent_cnn,env,epochs_test,prefix='models/cnn_test_t{}'.format(t), prt=False)
    print('Test of the FC - Temperature : {}'.format(t))
    test(agent_fc,env,epochs_test,prefix='models/fc_test_t{}'.format(t), prt=False)

Test of the CNN - Temperature : 0.3
Final score: 4.5
Test of the FC - Temperature : 0.3
Final score: -0.6363636363636364
Test of the CNN - Temperature : 0.6
Final score: 3.727272727272727
Test of the FC - Temperature : 0.6
Final score: 6.863636363636363
Test of the CNN - Temperature : 0.9
Final score: 5.954545454545454
Test of the FC - Temperature : 0.9
Final score: 16.40909090909091


**It appears that the FC DQN performs a lot better than the CNN DQN for high temperatures. This might be the case because, while both tend to often get stuck in a loop, the FC DQN tends to explore more than the CNN DQN (perhaps less precisely which yields bad score for low temperatures), which pays off when there are more bonuses than maluses.**

**The two videos below show example test runs of the two model for a temperature of 0.9.**

In [279]:
print("Test example of the FC DQN with 0.9 temperature")
HTML(display_videos('videos/fc_test_t0.95.mp4'))

Test example of the FC DQN with 0.9 temperature


In [6]:
print("Test example of the CNN DQN with 0.9 temperature")
HTML(display_videos('videos/cnn_test_t0.95.mp4'))

Test example of the CNN DQN with 0.9 temperature


***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [252]:
def train_explore(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)
            
            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
        
class EnvironmentExploring(Environment):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1, incr_malus = False):
        super(EnvironmentExploring, self).__init__(grid_size=grid_size, 
                                                   max_time=max_time, 
                                                   temperature=temperature)
        self.malus_position = np.zeros_like(self.board)
        
    
    def act(self, action, train = False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = 0
        if train:
            reward = -self.malus_position[self.x, self.y]
        if incr_malus:
            # The malu of the position increases each time we visit it
            self.malus_position[self.x, self.y]+= 0.1
        reward = reward + self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        
        game_over = self.t > self.max_time
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over
    
    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))

        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0
        
        self.malus_position = np.zeros_like(self.board)
        
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]
        return state

In [253]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='videos/cnn_train_explore')
HTML(display_videos('videos/cnn_train_explore10.mp4'))

Epoch 000/100 | Loss 0.0221 | Win/lose count 7.5/28.60000000000012 (-21.10000000000012)
Epoch 001/100 | Loss 0.0273 | Win/lose count 11.0/21.500000000000057 (-10.500000000000057)
Epoch 002/100 | Loss 0.0192 | Win/lose count 8.0/21.500000000000046 (-13.500000000000046)
Epoch 003/100 | Loss 0.0079 | Win/lose count 15.5/16.499999999999968 (-0.999999999999968)
Epoch 004/100 | Loss 0.0164 | Win/lose count 8.0/17.399999999999984 (-9.399999999999984)
Epoch 005/100 | Loss 0.0116 | Win/lose count 10.0/16.59999999999997 (-6.5999999999999694)
Epoch 006/100 | Loss 0.0092 | Win/lose count 9.0/25.90000000000011 (-16.90000000000011)
Epoch 007/100 | Loss 0.0061 | Win/lose count 9.5/15.599999999999964 (-6.099999999999964)
Epoch 008/100 | Loss 0.0119 | Win/lose count 11.5/18.800000000000008 (-7.300000000000008)
Epoch 009/100 | Loss 0.0090 | Win/lose count 15.0/20.10000000000001 (-5.1000000000000085)
Epoch 010/100 | Loss 0.0153 | Win/lose count 13.0/18.500000000000007 (-5.500000000000007)
Epoch 011/100 |

Epoch 093/100 | Loss 0.0259 | Win/lose count 17.5/16.999999999999964 (0.5000000000000355)
Epoch 094/100 | Loss 0.0540 | Win/lose count 22.5/11.099999999999977 (11.400000000000023)
Epoch 095/100 | Loss 0.0233 | Win/lose count 19.0/14.399999999999974 (4.600000000000026)
Epoch 096/100 | Loss 0.0282 | Win/lose count 19.5/14.499999999999972 (5.000000000000028)
Epoch 097/100 | Loss 0.0396 | Win/lose count 16.5/15.999999999999966 (0.5000000000000338)
Epoch 098/100 | Loss 0.0389 | Win/lose count 20.0/14.899999999999975 (5.1000000000000245)
Epoch 099/100 | Loss 0.0418 | Win/lose count 21.5/15.29999999999998 (6.200000000000021)


In [254]:
# Evaluation
test(agent,env,epochs_test,prefix='videos/cnn_test_explore')
HTML(display_videos('videos/cnn_test_explore10.mp4'))

Win/lose count 10.0/3.0. Average score (7.0)
Win/lose count 14.0/9.0. Average score (6.0)
Win/lose count 16.5/5.0. Average score (7.833333333333333)
Win/lose count 24.5/10.0. Average score (9.5)
Win/lose count 21.0/4.0. Average score (11.0)
Win/lose count 15.5/6.0. Average score (10.75)
Win/lose count 12.5/8.0. Average score (9.857142857142858)
Win/lose count 23.0/4.0. Average score (11.0)
Win/lose count 19.0/5.0. Average score (11.333333333333334)
Win/lose count 22.5/4.0. Average score (12.05)
Win/lose count 10.5/8.0. Average score (11.181818181818182)
Final score: 11.181818181818182


**This new environment successfully incentivizes the agent to explore more the board, which significantly increases the performances of the agent.**